In [1]:
import os

In [2]:
%pwd

'/Users/udita19/Desktop/Projects/BrainTumor/BrainTumorClassification/research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'/Users/udita19/Desktop/Projects/BrainTumor/BrainTumorClassification'

In [5]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    non_tumor_source_URL: str
    golima_source_URL: str

    non_tumor_local_file: Path
    golima_local_file: Path
    unzip_dir: Path
    

In [6]:
import sys
sys.path.append('/Users/udita19/Desktop/Projects/BrainTumor/BrainTumorClassification/src')


from TumorClassifier.constants import*
from TumorClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH) :
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig :
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir = config.root_dir,
        non_tumor_source_URL = config.non_tumor_source_URL,
        golima_source_URL = config.golima_source_URL,
        non_tumor_local_file = config.non_tumor_local_file,
        golima_local_file = config.golima_local_file,
        unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


    

In [8]:
import os 
import urllib.request as request
import zipfile 
from TumorClassifier import logger 
from TumorClassifier.utils.common import get_size


In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config




    def download_file(self):
        if not os.path.exists(self.config.non_tumor_local_file):
            filename, headers = request.urlretrieve(
                self.config.non_tumor_source_URL, 
                self.config.non_tumor_local_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.non_tumor_local_file))}")

        if not os.path.exists(self.config.golima_local_file):
            filename, headers = request.urlretrieve(
                self.config.golima_source_URL,
                self.config.golima_local_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
               logger.info(f"File already exists of size : {get_size(Path(self.config.golima_local_file))}")





    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok = True)

        with zipfile.ZipFile(self.config.golima_local_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        with zipfile.ZipFile(self.config.non_tumor_local_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [14]:
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-02-28 23:01:03,380: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-28 23:01:03,381: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-28 23:01:03,382: INFO: common: created directory at: artifacts]
[2024-02-28 23:01:03,382: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-28 23:01:03,383: INFO: 1497186472: File already exists of size : ~ 23151 KB]
[2024-02-28 23:01:09,648: INFO: 1497186472: artifacts/data_ingestion/golima.zip download! with following info: 
Connection: close
Content-Length: 24783689
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3dc6acdfe08248a5599ed2454e93f024d0b9be3742ac427ec0513ff6b32adc4f"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3560:20882F:559200:6E7F6B:65DF6DD7
Accept-Ranges: bytes
Date: 